In [30]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
#sqlcontext=SQLContext(sc)

In [31]:
poll=spark.createDataFrame([(1,"pollingtilte","2020","2021","pti","people")]).\
toDF("poll_id","title","startDate","endDate","organizer","participants")

In [32]:
question=spark.createDataFrame([(0,"questiontitle","Single Choice",1)]).\
toDF("question_id","title","type","poll")

In [33]:
organizer=spark.createDataFrame([(1,"gmail.com","robert","bing","01234")]).\
toDF("organizer_id","email","firstname","lastname","phone")

In [50]:
question.select("question_id","title","type","poll").filter(question["type"]=="Single Choice").join(poll,question.poll==poll.poll_id,"outer")\
.join(organizer,organizer["organizer_id"]==poll["organizer"],"left_anti").explain()

== Physical Plan ==
SortMergeJoin [cast(organizer#750 as bigint)], [organizer_id#784L], LeftAnti
:- *(5) Sort [cast(organizer#750 as bigint) ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(cast(organizer#750 as bigint), 200), ENSURE_REQUIREMENTS, [id=#1335]
:     +- SortMergeJoin [poll#769L], [poll_id#746L], FullOuter
:        :- *(2) Sort [poll#769L ASC NULLS FIRST], false, 0
:        :  +- Exchange hashpartitioning(poll#769L, 200), ENSURE_REQUIREMENTS, [id=#1325]
:        :     +- *(1) Project [_1#758L AS question_id#766L, _2#759 AS title#767, _3#760 AS type#768, _4#761L AS poll#769L]
:        :        +- *(1) Filter (isnotnull(_3#760) AND (_3#760 = Single Choice))
:        :           +- *(1) Scan ExistingRDD[_1#758L,_2#759,_3#760,_4#761L]
:        +- *(4) Sort [poll_id#746L ASC NULLS FIRST], false, 0
:           +- Exchange hashpartitioning(poll_id#746L, 200), ENSURE_REQUIREMENTS, [id=#1330]
:              +- *(3) Project [_1#734L AS poll_id#746L, _2#735 AS title#747, _3